In [ ]:
import openmm
import openmm.app as app
import openmm.unit as unit
import sys

# Load AMBER files+
prmtop = app.AmberPrmtopFile('P_4.prmtop')
inpcrd = app.AmberInpcrdFile('P_4.inpcrd')


# Create the system with nonbonded interactions and periodic boundary conditions +
#for the nonbonded method in the article SPME (smooth particle mesh Ewald) is used. The difference between SPME and PME is that SPME basically uses higher order, so the result is more precise. 
#OPENMM uses the SHAKE algorithm with constraints argument.
#in OPENMM By default, the constraint Tolerance is set to 1e-4 in the SHAKE algorithm. 

system = prmtop.createSystem(nonbondedMethod=app.PME, 
                             nonbondedCutoff=1.2*unit.nanometer, 
                             constraints=app.HBonds) 

#Define the Integrator Langevin
temperature = 0*unit.kelvin
friction = 1/unit.picosecond
timestep = 2*unit.femtosecond
integrator = openmm.LangevinIntegrator(temperature, friction, timestep)

# Add a barostat for constant pressure (1 atm)+
#coupling constant is given as 0.4 ps in the article.
pressure = 1*unit.atmosphere
barostat_frequency = 25  # How often volume adjustments are made
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, barostat_frequency))


#

# Create a simulation object+
#platform = Platform.getPlatformByName('CUDA')  # Use GPU if available
#properties = {'CudaPrecision': 'mixed'}
#platform = openmm.Platform.getPlatformByName('CUDA')
platform = openmm.Platform.getPlatformByName('CPU')  # Use CPU platform
simulation = app.Simulation(prmtop.topology, system, integrator, platform)

# Load initial positions+
simulation.context.setPositions(inpcrd.positions)

# If box vectors are provided in the inpcrd file, set them +
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

#------------------------------------------------------------------------------------------------
#Minimization-----------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------

# Minimization Based on the article, energy minimized based on conjugate gradient method.+
#OpenMM uses steepest descent as default method. 
#How can I decide max iterations?
print('Minimizing using Conjugate Gradient...')
simulation.minimizeEnergy(maxIterations=500)


# Save the minimized system (coordinates and topology)
minimized_pdb = 'minimized_step1.pdb'  # PDB file to save minimized coordinates

# Save the minimized coordinates to a PDB file
with open(minimized_pdb, 'w') as pdb_file:
    app.PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), pdb_file)


print("Minimization complete. File saved: minimized_step1.pdb")